In [21]:
start_date = "1960-01-01"
end_date = "2022-12-31"
import pandas_datareader as pdr
import pandas as pd
import numpy as np
from compustat_connection import Computstat
factors_ff3_monthly_raw = pdr.DataReader(
	name="F-F_Research_Data_Factors",
	data_source="famafrench",
	start=start_date,
	end=end_date)[0]
factors_ff3_monthly = (factors_ff3_monthly_raw
	.divide(100)
	.reset_index(names="month")
	.assign(
	month = lambda x: pd.to_datetime(x["month"].astype(str))
)
.rename(str.lower, axis="columns")
.rename(columns = {"mkt-rf" : "mkt_excess"})
)

factors_ff3_monthly

,month,mkt_excess,smb,hml,rf
0,1960-01-01,-0.0698,0.0209,0.0278,0.0033
1,1960-02-01,0.0117,0.0051,-0.0193,0.0029
2,1960-03-01,-0.0163,-0.0049,-0.0294,0.0035
3,1960-04-01,-0.0171,0.0032,-0.0228,0.0019
4,1960-05-01,0.0312,0.0121,-0.0370,0.0027
...,...,...,...,...,...
751,2022-08-01,-0.0377,0.0140,0.0029,0.0019
752,2022-09-01,-0.0935,-0.0081,0.0005,0.0019
753,2022-10-01,0.0783,0.0006,0.0801,0.0023
754,2022-11-01,0.0460,-0.0352,0.0138,0.0029


In [18]:
crsp_monthly_query = (
  "SELECT msf.permno, msf.date, "
          "date_trunc('month', msf.date)::date as month, "
          "msf.ret, msf.shrout, msf.altprc, "
          "msenames.exchcd, msenames.siccd, "
          "msedelist.dlret, msedelist.dlstcd "
    "FROM crsp.msf AS msf "
    "LEFT JOIN crsp.msenames as msenames "
    "ON msf.permno = msenames.permno AND "
       "msenames.namedt <= msf.date AND "
       "msf.date <= msenames.nameendt "
    "LEFT JOIN crsp.msedelist as msedelist "
    "ON msf.permno = msedelist.permno AND "
       "date_trunc('month', msf.date)::date = "
       "date_trunc('month', msedelist.dlstdt)::date "
    "WHERE msf.date BETWEEN '01/01/1960' AND '12/31/2022' "
          "AND msenames.shrcd IN (10, 11)"
) 

SELECT msf.permno, msf.date, date_trunc('month', msf.date)::date as month, msf.ret, msf.shrout, msf.altprc, msenames.exchcd, msenames.siccd, msedelist.dlret, msedelist.dlstcd FROM crsp.msf AS msf LEFT JOIN crsp.msenames as msenames ON msf.permno = msenames.permno AND msenames.namedt <= msf.date AND msf.date <= msenames.nameendt LEFT JOIN crsp.msedelist as msedelist ON msf.permno = msedelist.permno AND date_trunc('month', msf.date)::date = date_trunc('month', msedelist.dlstdt)::date WHERE msf.date BETWEEN '01/01/1960' AND '12/31/2022' AND msenames.shrcd IN (10, 11)


In [29]:
curr = Computstat('nathanielyee')
print(curr)


Loading library list...
Done
nathanielyee


In [30]:
crsp_df = curr.execute(crsp_monthly_query)


In [35]:
crsp_df['shrout_altprc'] = abs((crsp_df['shrout'] * crsp_df['altprc'])/1000000)
crsp_df['shrout_altprc'].replace(0, np.nan, inplace=True)
crsp_df


,permno,date,month,ret,shrout,altprc,exchcd,siccd,dlret,dlstcd,shrout_altprc
0,10000.0,1986-01-31,1986-01-01,NaN,3680.0,-4.375000,3.0,3990.0,NaN,NaN,0.016100
1,10000.0,1986-02-28,1986-02-01,-0.257143,3680.0,-3.250000,3.0,3990.0,NaN,NaN,0.011960
2,10000.0,1986-03-31,1986-03-01,0.365385,3680.0,-4.437500,3.0,3990.0,NaN,NaN,0.016330
3,10000.0,1986-04-30,1986-04-01,-0.098592,3793.0,-4.000000,3.0,3990.0,NaN,NaN,0.015172
4,10000.0,1986-05-30,1986-05-01,-0.222656,3793.0,-3.109375,3.0,3990.0,NaN,NaN,0.011794
...,...,...,...,...,...,...,...,...,...,...,...
431267,90251.0,2006-07-31,2006-07-01,-0.012759,135693.0,28.629999,1.0,3674.0,NaN,NaN,3.884890
431268,90251.0,2006-08-31,2006-08-01,0.072302,135693.0,30.700001,1.0,3674.0,NaN,NaN,4.165775
431269,90251.0,2006-09-29,2006-09-01,0.239414,143374.0,38.049999,1.0,3674.0,NaN,NaN,5.455381
431270,90251.0,2006-10-31,2006-10-01,0.034691,142231.0,39.369999,1.0,3674.0,NaN,NaN,5.599634
